In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

torch.manual_seed(1)


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 3.8 MB/s 
     |████████████████████████████████| 7.6 MB 70.4 MB/s 
     |████████████████████████████████| 182 kB 83.8 MB/s 


In [5]:
foldername= "/content/drive/My Drive/nlpproject/"
path = foldername+"dataset2.csv"

In [6]:
df = pd.read_csv(path).dropna()
items = df.iloc[:]["post"]
old_word_to_ix = dict()
for i,sent in enumerate(items):
    for word in sent.split():
        if word not in old_word_to_ix: 
            old_word_to_ix[word] = 1
        else:
            old_word_to_ix[word] = old_word_to_ix[word]+1
# Add unknown word "UNK" to word_to_ix.
# Doing this in case you find an unknown word in testing.
word_to_ix = {}
for word in old_word_to_ix:
  count=old_word_to_ix[word]
  if count>10:
    word_to_ix[word]=len(word_to_ix)
word_to_ix["UNK"] = len(word_to_ix)
print(len(word_to_ix))

33842


In [7]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")
embedding_matrix = model.embeddings.word_embeddings.weight

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
embedding_matrix

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

In [9]:
em=nn.Embedding(num_embeddings=30522, embedding_dim=768)
em.weight=embedding_matrix

In [10]:
em.weight.requires_grad=False

In [11]:
em.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]])

In [12]:
class CFG():
  max_len = 512
  model = "bert-base-uncased"
  lr=1e-4
  min_lr=1e-10
  batch_size=4096
  epoch=10000
  hidden_dim=512
  drop=0.15
cfg=CFG()

In [13]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained(cfg.model)
class Ds(Dataset):
    def __init__(self,path, id_mbti, tokenizer=tokenizer, max_token_len=50):
        self.df = pd.read_csv(path).dropna()
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.id_mbti=0  
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.id_mbti]
        if letter in "ESTJ":
            return 1
        else:
            return 0
    def label2str(self, label):
        return self.labelstrdicts[label][self.id_mbti]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [14]:
from transformers import DataCollatorWithPadding
path=foldername+"dataset2.csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    total_len=len(ds)
    train_len=int(len(ds)*0.9)
    val_len=int((total_len-train_len)/2)
    test_len=total_len-train_len-val_len
    [train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    return len(train_ds), len(val_ds), len(test_ds), DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    #return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [15]:
ds=Ds(path,0)
len_train, len_val, len_test, train_dl_EI,val_dl_EI, test_dl_EI=getdl(ds)

In [16]:
#l=[x['labels'] for x in ds]
#num_one=torch.count_nonzero(torch.tensor(l), dim=0)
#num_zero=len(l)-num_one

#print(num_zero)
#print(num_one)
LABEL_RATIO=torch.tensor([1.0000, 3.2796]).to("cuda")
print(LABEL_RATIO)
print(len(ds))

tensor([1.0000, 3.2796], device='cuda:0')
396523


In [17]:
def evaluate(labels, outputs):
  answers=(torch.argmax(outputs, dim=1))
  allcorrect=torch.sum(answers==labels)
  return allcorrect

In [19]:
from torch.nn.init import xavier_normal
class Model(nn.Module):

    def __init__(self,hidden_dim, drop):
        super(Model, self).__init__()

        self.word_embeddings=nn.Embedding(num_embeddings=30522, embedding_dim=768)
        self.word_embeddings.weight=embedding_matrix
        self.word_embeddings.weight.requires_grad=False

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(768, hidden_dim)
        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(drop)
        # The linear layer that maps from hidden state space to tag space
        self.fc1= nn.Linear(hidden_dim, 128)
        self.fc2= nn.Linear(128, 32)
        self.fc3= nn.Linear(32, 8)
        self.fc_final= nn.Linear(8, 2)

        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
        self._init_weights(self.fc3)
        self._init_weights(self.fc_final)

    def _init_weights(self, module):
          module.weight.data = nn.init.xavier_uniform_(module.weight.data)
          if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, input_ids):
        o =  self.word_embeddings(input_ids)
        o,_= self.lstm(o)
        o = self.dropout(o)
        o=o[:,-1]
        o = self.fc1(o)
        o = self.relu(o)
        o = self.dropout(o)
        o = self.fc2(o)
        o = self.relu(o)
        o = self.dropout(o)
        o = self.fc3(o)
        o = self.relu(o)
        o = self.dropout(o)
        o = self.fc_final(o)
        return o

In [20]:
from torch.optim import lr_scheduler
from torch.nn import CrossEntropyLoss
from torch import optim
from torch.optim import Adam
from tqdm.notebook import tqdm
def train(train_ds, eval_ds, model, epochs, cfg, type, lr, loss=None):
    if torch.cuda.is_available():  
        dev = "cuda:0" 
    else:  
        dev = "cpu" 
    device = torch.device(dev)
    model = model.to(device)

    #weights=torch.tensor([1., 3.]).cuda()
    #criterion = nn.MSELoss()
    criterion = CrossEntropyLoss(weight=LABEL_RATIO)

    criterion.to(device)
    #criterion = loss
    
    optimizer = Adam(model.parameters(), lr=lr)
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    #scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=500, eta_min=1e-15)
    scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)
        
    totalevalloss=0
    totalcorrect=0
    totaldata=0
    with torch.no_grad():
        model.eval()
        for batch in eval_ds:
            batch.to(device)
            blabels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            eloss=criterion(outputs, blabels).item()
            totalevalloss+=eloss
            totalcorrect+=evaluate(blabels, outputs)
            totaldata+=len(blabels)
    totalcorrect_rate=(totalcorrect/(totaldata))
    print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
    #print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} ' ) 
    print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} | current lr: {scheduler.get_last_lr()}' ) 
    #print(f'Initial Val Loss: {totalevalloss: .3f} | current lr: {scheduler.get_last_lr()}' ) 
    
    for e in range(epochs):
        totaltrainloss=0
        totaltraincorrect=0
        totaltraindata=0
        model.train()
        for i,batch in enumerate(train_ds):

            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            optimizer.step()
            optimizer.zero_grad()
            totaltrainloss+=bloss.item()
            totaltraincorrect+=evaluate(labels, outputs)
            totaltraindata+=len(labels)
        scheduler.step()
        totalevalloss=0
        totalcorrect=0
        totaldata=0
        with torch.no_grad():
            model.eval()
            for batch in eval_ds:
                batch.to(device)
                blabels=batch["labels"]
                outputs=model(input_ids=batch["input_ids"])
                eloss=criterion(outputs, blabels).item()
                totalevalloss+=eloss
                totalcorrect+=evaluate(blabels, outputs)
                totaldata+=len(blabels)
        totalcorrect_rate=(totalcorrect/(totaldata))
        totaltraincorrect_rate = (totaltraincorrect/(totaltraindata))
        print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
        print("probability that our prediction of ", type, " is correct in training dataset: ", totaltraincorrect_rate)
        #print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .3f}' ) 
        print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .8f} | current lr: {scheduler.get_last_lr()}' ) 
        #print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss: .8f} | Val Loss: {totalevalloss: .8f} | current lr: {scheduler.get_last_lr()}' ) 

In [ ]:
model = Model(cfg.hidden_dim, drop=0.05)
train(train_dl_EI, val_dl_EI, model, epochs=cfg.epoch, cfg=cfg, type="EI", lr=1e-4, loss=None)

probability that our prediction of  EI  is correct:  tensor(0.2333, device='cuda:0')
Initial Val Loss:  6.932 | current lr: [0.0001]
probability that our prediction of  EI  is correct:  tensor(0.5689, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5411, device='cuda:0')
Epoch: 1 | Train Loss:  121.30362749 | Val Loss:  6.93141711 | current lr: [9.900000000000001e-05]
probability that our prediction of  EI  is correct:  tensor(0.4503, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5373, device='cuda:0')
Epoch: 2 | Train Loss:  121.29740226 | Val Loss:  6.93154722 | current lr: [9.801e-05]
probability that our prediction of  EI  is correct:  tensor(0.5689, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5379, device='cuda:0')
Epoch: 3 | Train Loss:  121.29565179 | Val Loss:  6.93137413 | current lr: [9.70299e-05]
probability that our p

In [31]:
model = Model(cfg.hidden_dim, drop=0.05)
model.to("cuda")

Model(
  (word_embeddings): Embedding(30522, 768)
  (lstm): LSTM(768, 512)
  (relu): ReLU()
  (dropout): Dropout(p=0.05, inplace=False)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
  (fc_final): Linear(in_features=8, out_features=2, bias=True)
)

In [32]:
criterion = CrossEntropyLoss()
print(model)
model.train()
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)
device="cuda"
for e in range(100):
    x=0
    for batch in train_dl_EI:
            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            x=x+bloss.item()
            optimizer.step()
            optimizer.zero_grad()
            print(model.fc1.weight)
            scheduler.step()
            break
    print(x)

Model(
  (word_embeddings): Embedding(30522, 768)
  (lstm): LSTM(768, 512)
  (relu): ReLU()
  (dropout): Dropout(p=0.05, inplace=False)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
  (fc_final): Linear(in_features=8, out_features=2, bias=True)
)
Parameter containing:
tensor([[ 0.0780, -0.0958, -0.0576,  ..., -0.0166,  0.0612,  0.0188],
        [-0.0217,  0.0083, -0.0455,  ..., -0.0513, -0.0551, -0.0967],
        [ 0.0725, -0.0666, -0.0327,  ...,  0.0424,  0.0091,  0.0918],
        ...,
        [-0.0512, -0.0418,  0.0458,  ..., -0.0312, -0.0410, -0.0848],
        [ 0.0135,  0.0880, -0.0079,  ...,  0.0643,  0.0143,  0.0665],
        [-0.0913,  0.0099,  0.0529,  ...,  0.0136,  0.0336,  0.0283]],
       device='cuda:0', requires_grad=True)
0.695945680141449
Parameter containing:
tensor([[ 0.0781, -0.0957, -0.0577,  ..., -0.0167,  0.0611,  0.0187],
       

KeyboardInterrupt: ignored